In [14]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from nameparser import HumanName

In [3]:
url = 'https://www.helptheheroes.net/In-Memoriam'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
pars = soup.find_all('h4')

In [5]:
def get_states():
    states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
              "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois",
              "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland",
              "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana",
              "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York",
              "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
              "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah",
              "Puerto Rico", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
    return (states)

def one_entry(paragraph, states):
    if paragraph.text == "\xa0":
        print('Empty')
    link = 'None'
    name = 'None'
    age = 'None'
    specialty = 'None'
    city = 'None'
    state = 'None'
    country = 'None'
    location = 'None'
    try:
        link = paragraph.find('a')['href']
    except TypeError:
        link = 'None'
    try:
        info_text = paragraph.text.replace('\n', '').replace('\xa0', '')
        info_list = info_text.split(', ')
        info_update = []
        for info in info_list:
            info_update += (info.split(','))
        info_list = info_update
        name = info_list[0]
        if len(info_list) == 4:
            age = info_list[1]
            specialty = info_list[2]
            city = info_list[3]
        else:
            for info in info_list:
                info = info.replace(' (presumed)', '')
                if info.isnumeric() or info == 'age unknown':
                    age = info
                if info in states:
                    state = info
                    country = 'USA'
                    city = info_list[-2]
            if country != 'USA':
                country = info_list[-1].replace(' (presumed)', '')
                city = info_list[-2]
            if age != 'None':
                remain = info_text.split(f'{age},')[1]
                remainder = remain.replace(f'{city}, ', '').replace(state, '')
                remainder = remainder.replace(country, '')
                if remainder[0] == ' ':
                        remainder = remainder[1:]
                specialty = remainder.replace(', ', '  ')
                remainder_list = remainder.split(', ')
                if len(remainder_list) > 2:
                    location = remainder_list[-2]
                    specialty = remainder.replace(location, '').replace(', ', ' ')
            specialty = specialty.replace('  ', '')
            if len(specialty) > 1:
                if specialty[-1] == ' ':
                    specialty = specialty[:-1]
            if country == 'New York City':
                city = 'New York City'
            if city == 'New York City':
                country = 'USA'
                state = 'New York'
            if city == 'Washington':
                state = 'DC'
                specialty = specialty.replace(',DC', '')
            if city == 'age unknown':
                city = 'None'
            if country == 'None':
                country = city
                city = 'None'
    except IndexError as index_error:
        print('Human intervention needed for the following exception:')
        print(index_error)
        print(paragraph.text)
        print(' ')
    new_dict = {
            'NAME': name,
            'AGE': age,
            'SPECIALTY': specialty,
            'CITY': city,
            'STATE': state,
            'COUNTRY': country,
            'LOCATION': location,
            'LINK': link
        }
    return new_dict

def grab_data(data):
    no_link_count = 0
    states = get_states()
    dict_list = []
    for paragraph in data[1:]:
        new_dict = one_entry(paragraph, states)
        dict_list.append(new_dict)
    return dict_list

In [6]:
df = pd.DataFrame(grab_data(pars))

Human intervention needed for the following exception:
list index out of range

 


In [8]:
PPD = pd.read_csv("../../Data/PPD/ppd_data_20200404.csv")

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
tweets = pd.read_csv("../../Data/Medscape/tweets.csv")
all_tweets = pd.read_csv("../../Data/Medscape/ALL_TWEETS.csv")

In [9]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums

In [110]:
def append_me(roster_df):
    #Matches to PPD and appends ME
    bad_spec_words = [
        'NURS',
        'VET',
        'TRANSPORT',
        'ASSISTANT',
        'RECEPTIONIST',
        'TECHNICIAN',
        'PARAMEDIC',
        'AIDE',
        'SOCIAL WORKER',
        'ENTREPRENEUR',
        'SERVICES',
        'GROUPHOME',
        'RN',
        'SECURITY',
        'PHARMACIST',
        'FIRE',
        'EMPLOYEE',
        'DEVELOPER',
        'PSYCHIATRIST',
        'ADMINISTRATOR',
        'LEADER',
        'LPN',
        'THERAPIST',
        'CLERK',
        'COUNSELOR',
        'ATTENDANT',
        'ADMIN',
        'SUPPLY',
        'CLEAN',
        'PRIEST',
        'STAFF',
        'INVESTIGATOR',
        'MRI',
        'EDUCATOR',
        'OFFICER',
        'MAINTENANCE',
        'CNA',
        'SUPERVISOR',
        'COORDINATOR',
        'SUPERVISOR',
        'TECHNOLOGIST',
        'MECHANIC'
    ]
    from_twitter = False
    if 'DATE' in roster_df.columns:
        from_twitter=True
        data_split = roster_df
    else:
        data_split = split_names(roster_df)
    mes = []
    for row in data_split.itertuples():
        years=False
        physician_me = 'None'
        keep = True
        if from_twitter == False:
            for word in bad_spec_words:
                if word in row.SPECIALTY.upper():
                    keep = False
            for thing in row.CITY.split(' '):
                if thing.isnumeric():
                    keep = False
        if keep:
            try:
                years = [2019.0 - int(row.AGE), 2020.0 - int(row.AGE)]
            except ValueError:
                pass
            new_df = PPD[(PPD.FIRST_NAME == row.FIRST_NAME) & (PPD.LAST_NAME == row.LAST_NAME)]
            if len(new_df) == 0 and years:
                new_df = PPD[(PPD.LAST_NAME == row.LAST_NAME) & (PPD.BIRTH_YEAR.isin(years))]
                if len(new_df) == 0:
                    pass
                if len(new_df) > 1:
                    if from_twitter==False:
                        new_df = new_df[new_df.CITY == row.CITY.upper()]
                    else:
                        new_df = new_df[new_df.STATE == row.STATE]
            elif len(new_df) > 1 and years:
                new_df = new_df[new_df.BIRTH_YEAR.isin(years)]
                if len(new_df) > 1:
                    print('wtf')
                    print(row.NAME)
            if len(new_df) == 1:
                physician_me = list(new_df.ME)[0]

        mes.append(physician_me)

    data_split['ME'] = fix_me(mes)
    data_me = data_split[data_split.ME != 'None']
    return data_split, data_me

In [111]:
data_split, me_df = append_me(df)

wtf
Michael Giuliano
wtf
Priya Khanna


In [ ]:
split, hero_me_df = append_me(df)

In [93]:
    bad_spec_words = [
        'NURS',
        'VET',
        'TRANSPORT',
        'ASSISTANT',
        'RECEPTIONIST',
        'TECHNICIAN',
        'PARAMEDIC',
        'AIDE',
        'SOCIAL WORKER',
        'ENTREPRENEUR',
        'SERVICES',
        'GROUPHOME',
        'RN',
        'SECURITY',
        'PHARMACIST',
        'FIRE',
        'EMPLOYEE',
        'DEVELOPER',
        'PSYCHIATRIST',
        'ADMINISTRATOR',
        'LEADER',
        'LPN',
        'THERAPIST',
        'CLERK',
        'COUNSELOR',
        'ATTENDANT',
        'ADMIN',
        'SUPPLY',
        'CLEAN',
        'PRIEST',
        'STAFF',
        'INVESTIGATOR',
        'MRI',
        'EDUCATOR',
        'OFFICER',
        'MAINTENANCE',
        'CNA',
        'SUPERVISOR',
        'COORDINATOR',
        'SUPERVISOR',
        'TECHNOLOGIST',
        'MECHANIC'
    ]

In [94]:
for thing in list(data_split['SPECIALTY']):
    keep = True
    for word in bad_spec_words:
        if word in thing.upper():
            keep=False
    if thing=='None':
        keep=False
    if keep==True:
        print(thing)

General Practitioner
Family Medicine
Medical Director
Emergency Department Medical Director
OB-GYN
Chief and Training Director
Holy Cross Hospital
Director of Radiology
Health and Safety Director at Christ Child House
Madera
Phlebotomist
Family Practitioner
Primary Care Physician
Brooklyn
PGY6 Oral-Maxillofacial Surgery Chief Resident
Emergency Medicine PhysicianUnited States
Ophthamologist
Doctor
Neurosurgeon Director of the Division of Pediatric Neurosurgery at Montefiore Health System Professor of Clinical Neurological Surgery PediatricsUnited States
Pediatrics
Orthodonist
New York
Bowling Green
NephrologistKhanna Nephrology,
Doctor Pulmonary/Critical Care Medicine
Hackensack
Chicago
Pulmonary and Critical Care Physician
Muskegon
Family Physician,Henry Ford Medical Group
Annandale

Urologist
Jamaica
Pediatrician,
Pathologist
Neurologist
Putnam County
Burien
Kings County
Chicago
Trauma Surgeon
 NP in West Bloomfield
General Surgery
St. Louis
General Surgery
Physician,Electrophysiolog

In [58]:
len(me_df)

64

In [28]:
PPD[PPD['FIRST_NAME']=='SATYAVARDHANA']

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST


In [12]:
def split_names(roster_df):
    #Splits name column into components
    roster_df = roster_df.drop_duplicates()
    dict_list = []
    for row in roster_df.itertuples():
        name_parsed = HumanName(row.NAME)
        name_dict = {
            'NAME': row.NAME,
            'FIRST_NAME': name_parsed.first.upper(),
            'LAST_NAME': name_parsed.last.upper(),
            'MIDDLE_NAME': name_parsed.middle.upper(),
            'SUFFIX': name_parsed.suffix.upper(),
            'NICKNAME': name_parsed.nickname.upper(),
            'TITLE': name_parsed.title.upper(),
        }
        dict_list.append(name_dict)
    name_df = pd.DataFrame(dict_list)
    new_df = pd.merge(name_df, roster_df, on='NAME')
    return new_df

In [47]:
x, y = append_me(tweets)

wtf
Dr. Michael Giuliano
wtf
Dr. James Wilson


In [61]:
y.drop_duplicates('Name')

,Name,FIRST_NAME,LAST_NAME,MIDDLE_NAME,SUFFIX,NICKNAME,TITLE,Age,Date,State,Link,AGE,DATE,STATE,LINK,ME
4,Dr. Michael Medici,MICHAEL,MEDICI,NaN,NaN,NaN,DR.,78,2020-06-13 13:40:09+00:00,CA,https://wwwgooglecom/amp/s/wwwlegacycom/obitua...,NaN,NaN,NaN,NaN,01002670662
6,Dr. Stephan Kamholz,STEPHAN,KAMHOLZ,NaN,NaN,NaN,DR.,72,2020-06-12 03:01:57+00:00,None,https://boropark24com/news/dr-stephan-kamholz-...,NaN,NaN,NaN,NaN,03509720559
9,Dr. Robert “Ray” Hull,ROBERT,HULL,“RAY”,NaN,NaN,DR.,NaN,NaN,NaN,NaN,78,2020-06-11 16:16:00+00:00,AR,https://wwwgooglecom/amp/s/katvcom/amp/news/lo...,04706711649
11,Dr. Herbert Henderson Jr,HERBERT,HENDERSON,NaN,JR,NaN,DR.,NaN,NaN,NaN,NaN,56,2020-06-10 23:54:00+00:00,SC,https://wwwfletcherfuneralhomenet/obituary/DrH...,04501940441
16,Dr. Chaihan Korn,CHAIHAN,KORN,NaN,NaN,NaN,DR.,None,2020-06-07 14:31:00+00:00,AL,https://wwwgooglecom/amp/s/wwwwbrccom/2020/06/...,NaN,NaN,NaN,NaN,89104710094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Dr. Ashraf Metwally,ASHRAF,METWALLY,NaN,NaN,NaN,DR.,None,2020-04-05 11:59:10+00:00,None,None,NaN,NaN,NaN,NaN,91503870156
206,Dr. Mark Allen Respler,MARK,RESPLER,ALLEN,NaN,NaN,DR.,66,2020-04-03 02:58:13+00:00,None,https://hamodiacom/2020/03/27/bde-dr-mark-alle...,NaN,NaN,NaN,NaN,03508801512
208,Dr. Frank Gabrin,FRANK,GABRIN,NaN,NaN,NaN,DR.,None,2020-04-01 13:19:36+00:00,None,None,NaN,NaN,NaN,NaN,04177851693
211,Dr. Stephen Schwartz,STEPHEN,SCHWARTZ,NaN,NaN,NaN,DR.,NaN,NaN,NaN,NaN,78,2020-04-01 01:38:07+00:00,None,None,02405670502


In [52]:
PPD[(PPD['LAST_NAME']=='SCHWARTZ')&(PPD['FIRST_NAME']=='STEPHEN')]['BIRTH_YEAR']

65521      1945.0
199204     1986.0
242235     1946.0
294041     1946.0
349101     1942.0
375816     1960.0
440110     1940.0
496552     1961.0
562776     1943.0
566825     1969.0
620171     1935.0
706064     1944.0
708078     1954.0
733313     1935.0
784195     1960.0
1123702    1946.0
Name: BIRTH_YEAR, dtype: float64

In [62]:
blep = y.drop_duplicates('Name')

In [57]:
y.columns

Index(['Name', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'NICKNAME',
       'TITLE', 'Age', 'Date', 'State', 'Link', 'AGE', 'DATE', 'STATE', 'LINK',
       'ME'],
      dtype='object')

In [65]:
pd.concat([blep, me_df]).drop_duplicates('ME').to_excel("../../Data/Medscape/new_sources.xlsx")

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [66]:
new = pd.concat([blep, me_df]).drop_duplicates('ME')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [73]:
import pyodbc
username = 'vigrose'
password = 'slytherin10946'
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

def get_processed_list():
    '''Get latest processed ME list from EDW'''
    sql_query = \
        """
        SELECT DISTINCT
        M.MED_EDU_NBR
        FROM
        AMAEDW.PERSON M
        WHERE
        M.MORTALITY_STS_CD ='C'
        OR
        M.MORTALITY_STS_CD ='P';
        """
    known_mortality_status = pd.read_sql(con=AMAEDW, sql=sql_query)

    processed_mes = list(known_mortality_status.MED_EDU_NBR)

    return processed_mes

def remove_processed_mes(physicians):
    '''Remove physicians whose ME numbers are already processed'''
    processed_list = get_processed_list()
    unprocessed_physicians = physicians[physicians.ME.isin(processed_list)==False]

    return unprocessed_physicians

boop = remove_processed_mes(new)

In [72]:
me_df.to_excel("../../Data/Medscape/heroes.xlsx")

In [74]:
boop.to_excel("../../Data/Medscape/unprocessed_phys.xlsx")

In [109]:
pd.merge(blep, me_df, on='ME', how='outer')

,Name,FIRST_NAME_x,LAST_NAME_x,MIDDLE_NAME_x,SUFFIX_x,NICKNAME_x,TITLE_x,Age,Date,State,...,SUFFIX_y,NICKNAME_y,TITLE_y,AGE_y,SPECIALTY,CITY,STATE_y,COUNTRY,LOCATION,LINK_y
0,Dr. Michael Medici,MICHAEL,MEDICI,NaN,NaN,NaN,DR.,78,2020-06-13 13:40:09+00:00,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dr. Stephan Kamholz,STEPHAN,KAMHOLZ,NaN,NaN,NaN,DR.,72,2020-06-12 03:01:57+00:00,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dr. Robert “Ray” Hull,ROBERT,HULL,“RAY”,NaN,NaN,DR.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dr. Herbert Henderson Jr,HERBERT,HENDERSON,NaN,JR,NaN,DR.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dr. Chaihan Korn,CHAIHAN,KORN,NaN,NaN,NaN,DR.,None,2020-06-07 14:31:00+00:00,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,37,PGY6 Oral-Maxillofacial Surgery Chief Resident,Detroit,None,Michigan,None,https://www.detroitnews.com/story/business/202...
77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,73,Neurosurgeon Director of the Division of Pedia...,New York,New York,USA,Plastic and Reconstructive Surgery at the Albe...,https://www.nytimes.com/2020/04/01/obituaries/...
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,None,None,75 year old Dietary Services employee at Attle...,None,Pennsylvania,None,https://twitter.com/ClaireZebra/status/1262188...
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,92,Pulmonary and Critical Care Physician,San Francisco,California,USA,San Francisco General Hospital,https://www.mercurynews.com/2020/03/25/longtim...


In [108]:
me_df

,NAME,FIRST_NAME,LAST_NAME,MIDDLE_NAME,SUFFIX,NICKNAME,TITLE,AGE,SPECIALTY,CITY,STATE,COUNTRY,LOCATION,LINK,ME
1,Ashraf Abdo,ASHRAF,ABDO,,,,,60,General Practitioner,New York City,None,None,None,https://www.egypttoday.com/Article/1/83243/Egy...,91502920371
6,Nancy Ajemian,NANCY,AJEMIAN,,,,,60,Family Medicine,Grosse Pointe,Michigan,USA,Beaumont Hospital,https://twitter.com/DexterCMason/status/124988...,42201850562
37,Lorna Breen,LORNA,BREEN,,,,,49,Emergency Department Medical Director,NewYork-Presbyterian Allen Hospital,None,New York City (died by suicide),None,https://www.nytimes.com/2020/04/27/nyregion/ne...,05104991558
60,Evelyn Caro,EVELYN,CARO,,,,,69,Holy Cross Hospital,Silver Spring,Maryland,USA,None,https://www.baltimoresun.com/coronavirus/bs-md...,74801684975
72,Mary Clements,MARY,CLEMENTS,,,,,None,None,66 yo RN in Mobile,Alabama,USA,None,"Mary Clements, 66 yo RN in Mobile Alabama",01902860297
75,Dr. John Coman,JOHN,COMAN,,,,DR.,None,None,71yo Internal Medicine physician in Queens,None,NY,None,https://m.facebook.com/login.php?_rdr&next=htt...,56101800321
97,Orlando Espinoza,ORLANDO,ESPINOZA,,,,,53,Family Practitioner,Broward County,Florida,USA,None,https://www.vitals.com/doctors/Dr_Orlando_Espi...,30813980637
102,Jessie Ferreras,JESSIE,FERRERAS,,,,,55,Primary Care Physician,Waldwick,New Jersey,USA,None,None,74801820887
106,Chris Firlit,CHRIS,FIRLIT,,,,,37,PGY6 Oral-Maxillofacial Surgery Chief Resident,Detroit,None,Michigan,None,https://www.detroitnews.com/story/business/202...,02507173333
109,Frank Gabrin,FRANK,GABRIN,,,,,60,Emergency Medicine PhysicianUnited States,New York,New York,USA,None,https://www.nj.com/coronavirus/2020/04/nj-er-d...,04177851693
